In [1]:
from src.Gates import QuantumGate, PauliGate, PauliExpression

In [2]:
gate_name = 'xy'
obj = PauliGate(name=gate_name)
obj.matrix()

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
       [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [7]:
exs = 'zz+zz+yy+zy+xy+xx'
obj = PauliExpression(expression=exs)
obj.matrix()

array([[ 2.+0.j,  0.-1.j,  0.+0.j,  0.-1.j],
       [ 0.+1.j, -2.+0.j,  2.+1.j,  0.+0.j],
       [ 0.+0.j,  2.-1.j, -2.+0.j,  0.+1.j],
       [ 0.+1.j,  0.+0.j,  0.-1.j,  2.+0.j]], dtype=complex64)